This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_NAME = 'quandl'
API_KEY = None

#Read API Key from an ignored directory to avoid cleaning up api keys with each commit
with open("../ignore/API_KEYS.txt","r") as file:
    try:
        api_dict = dict([_.strip('\n').split(":") for _ in file.readlines()]) #Placeholder. Make sure .txt conforms: lines should be of form 'key':'value'
    except:
        raise ValueError('File format error. Format should be key:value [\w:\w] for each line')
    API_KEY = api_dict.get(API_NAME,None)

if API_KEY is None:
    raise ValueError('API_KEY Not Available')

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), part of which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

**Note that data for Carl Zeiss was not available for free on Quandl, analyzing adidas AG (ADS) instead.**

**adidas AG designs, develops, produces and markets a broad range of atletic and sports lifestyle products.**
https://report.adidas-group.com/2019/en/servicepages/downloads/files/consolidated_financial_statements_adidas_ar19.pdf

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import datetime 
import json
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
#GET https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}

database_code = "XFRA"
dataset_code = "ADS" 
return_format = 'json'
url = 'https://www.quandl.com/api/v3/datasets/'+database_code+'/'+dataset_code+'/'+'data.'+return_format
params = {'api_key':API_KEY, 'limit':2}
res = requests.get(url, params)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

res_d = res.json()
print(type(res_d), '\n')
print(res_d, '\n')
print(res_d.keys(), '\n')
print(res_d['dataset_data'].keys())
print(res_d['dataset_data']['column_names'])
#'column names' are: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adjustment Factor', 'Adjustment Type']
#data can be found in res_json['dataset_data']['data']


<class 'dict'> 

{'dataset_data': {'limit': 2, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adjustment Factor', 'Adjustment Type'], 'start_date': '2007-01-02', 'end_date': '2018-11-30', 'frequency': 'daily', 'data': [['2018-11-30', 195.453927, 195.552592, 190.718042, 192.592663, 2702.0, None, None], ['2018-11-29', 198.315191, 198.315191, 195.009938, 195.207267, 259.0, None, None]], 'collapse': None, 'order': None}} 

dict_keys(['dataset_data']) 

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adjustment Factor', 'Adjustment Type']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

**1. Collect data from the Franfurt Stock Exchange, 
for the ticker ADS, for the whole year 2017 
(keep in mind that the date format is YYYY-MM-DD).**

In [5]:
database_code = "XFRA"
dataset_code = "ADS" 
return_format = 'json'
url = 'https://www.quandl.com/api/v3/datasets/'+database_code+'/'+dataset_code+'/'+'data.'+return_format
start_date = datetime.date(2017,1,1).strftime('%Y/%m/%d')
end_date = datetime.date(2017,12,31).strftime('%Y/%m/%d')



params_2017 = {'api_key':API_KEY, 'start_date':start_date, 'end_date':end_date}
res_2017 = requests.get(url, params_2017)

**2. Convert the returned JSON object into a Python dictionary.**

In [6]:
res_2017d = res_2017.json()
print(type(res_2017d))

<class 'dict'>


**Write functions to extract data lists and calculate median.**

In [7]:
def get_column(colname, quandl_data_dict):
    cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adjustment Factor', 'Adjustment Type']
    col_dict = dict(zip(cols, range(len(cols))))
    target_data = []
    for _ in quandl_data_dict['dataset_data']['data']:
        target_data.append(_[col_dict[colname]])
    return target_data

def median(data):
    if len(data) == 0:
        return None
    
    data.sort()
    if len(data)%2==1:
        return data[len(data)//2]
    else:
        return (data[len(data)//2] + data[len(data)//2-1])/2
assert median([4,3,5])==4
assert median([1,2,5,4])==3
        

**3. Calculate what the highest and lowest opening prices were 
for the stock in this period.**

In [8]:
open_data = get_column('Open',res_2017d)

max_open_px = max(open_data)
min_open_px = min(open_data)
print("Highest opening price in 2017 was {0:.2f}".format(max_open_px))
print("Lowest opening price in 2017 was {0:.2f}".format(min_open_px))

Highest opening price in 2017 was 193.97
Lowest opening price in 2017 was 138.78


**4. What was the largest change in any one day (based on High and Low price)?**

In [9]:
high_data = get_column('High', res_2017d)
low_data = get_column('Low', res_2017d)

largest_intraday_chg = max([abs(_[0]-_[1]) for _ in zip(high_data, low_data)])
print("Largest intraday price change in 2017 was {0:.2f}".format(largest_intraday_chg))

Largest intraday price change in 2017 was 11.08


5. What was the largest change between any two days (based on Closing Price)?

In [10]:
closing_data = get_column('Close', res_2017d)

max_oneday_px_move = max([abs(_[0]-_[1]) for _ in zip(closing_data[:-1], closing_data[1:])])
print("Largest one day closing price change in 2017 was {0:.2f}".format(max_oneday_px_move))

Largest one day closing price change in 2017 was 14.05


**6. What was the average daily trading volume during this year?**


According to the documentation, https://www.quandl.com/data/XFRA-Frankfurt-Stock-Exchange-Prices/documentation, volume is represented as 0 on market holidays. Our calculations should do their best to reflect that. Given that adidas AG is a large company and high-volume stock it's reasonable to assume that all 0 volume days are stock market holidays. I will make the calculation accordingly. //A brief internet search confirms 9 of these 12 no-volume days as holidays. For the purposes of this exercise will not search further and accept 12 the number of holidays/no-trading-days.

In [11]:
vol_data = get_column('Volume', res_2017d)
date_data = get_column('Date', res_2017d)

dt_vol_exholiday_data = [_ for _ in zip(date_data, vol_data) if _[1] > 0]
dt_vol_holiday_data = [_ for _ in zip(date_data, vol_data) if _[1] == 0]


print('There were {0} days of 0-volume days in 2017'.format(len(dt_vol_holiday_data)))
print('0-volume days are as follows:\n{0}'.format('\n'.join([_[0] for _ in dt_vol_holiday_data])))

vol_exhol_data = [_[1] for _ in dt_vol_exholiday_data]
avg_daily_vol = sum(vol_exhol_data)/len(dt_vol_exholiday_data)


There were 12 days of 0-volume days in 2017
0-volume days are as follows:
2017-12-26
2017-12-25
2017-10-31
2017-10-03
2017-09-05
2017-08-23
2017-06-28
2017-06-05
2017-05-01
2017-04-19
2017-04-17
2017-04-14


**7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)**

In [12]:
median_vol = median(vol_exhol_data)

print("Median daily trading volume in 2017 was {0:.2f}".format(median_vol))

Median daily trading volume in 2017 was 1242.50
